# PART 1
###### Getting the data and organzing it.

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [2]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
source = requests.get(url).text
Canada_data = BeautifulSoup(source, 'lxml')

In [3]:
column_names = ['Postalcode','Borough','Neighborhood']
toronto = pd.DataFrame(columns = column_names)

In [5]:
content = Canada_data.find('div', class_='mw-parser-output')
table = content.table.tbody
postcode = 0
borough = 0
neighborhood = 0

for tr in table.find_all('tr'):
    i = 0
    for td in tr.find_all('td'):
        if i == 0:
            postcode = td.text
            i = i + 1
        elif i == 1:
            borough = td.text
            i = i + 1
        elif i == 2: 
            neighborhood = td.text.strip('\n').replace(']','')
    toronto = toronto.append({'Postalcode': postcode,'Borough': borough,'Neighborhood': neighborhood},ignore_index=True)

# clean dataframe 
toronto = toronto[toronto.Borough!='Not assigned']
toronto = toronto[toronto.Borough!= 0]
toronto.reset_index(drop = True, inplace = True)
i = 0
for i in range(0,toronto.shape[0]):
    if toronto.iloc[i][2] == 'Not assigned':
        toronto.iloc[i][2] = toronto.iloc[i][1]
        i = i+1
                                 
df = toronto.groupby(['Postalcode','Borough'])['Neighborhood'].apply(', '.join).reset_index()
df.head()

,Postalcode,Borough,Neighborhood
0,M1A\n,Not assigned\n,
1,M1B\n,Scarborough\n,Malvern / Rouge
2,M1C\n,Scarborough\n,Rouge Hill / Port Union / Highland Creek
3,M1E\n,Scarborough\n,Guildwood / Morningside / West Hill
4,M1G\n,Scarborough\n,Woburn


In [8]:
df.shape

(180, 3)

In [11]:
df1 = df[df.Borough != 'Not assigned\n']
df1 = df1.sort_values(by=['Postalcode','Borough'])

df1.reset_index(inplace=True)
df1.drop('index',axis=1,inplace=True)

df1.head()

,Postalcode,Borough,Neighborhood
0,M1B\n,Scarborough\n,Malvern / Rouge
1,M1C\n,Scarborough\n,Rouge Hill / Port Union / Highland Creek
2,M1E\n,Scarborough\n,Guildwood / Morningside / West Hill
3,M1G\n,Scarborough\n,Woburn
4,M1H\n,Scarborough\n,Cedarbrae


In [74]:
df_postalcodes = df1['Postalcode']
df_postalcodes.drop_duplicates(inplace=True)
df2 = pd.DataFrame(df_postalcodes)
df2['Borough'] = '';
df2['Neighbourhood'] = '';


df2.reset_index(inplace=True)
df2.drop('index', axis=1, inplace=True)
df1.reset_index(inplace=True)
df1.drop('index', axis=1, inplace=True)

for i in df2.index:
    for j in df1.index:
        if df2.iloc[i, 0] == df1.iloc[j, 0]:
            df2.iloc[i, 1] = df1.iloc[j, 1]
            df2.iloc[i, 2] = df2.iloc[i, 2] + ',' + df1.iloc[j, 2]
            
for i in df2.index:
    s = df2.iloc[i, 2]
    if s[0] == ',':
        s =s [1:]
    df2.iloc[i,2 ] = s
print(df2.shape)
df2['Neighbourhood'] = df2['Neighbourhood'].str.replace('/',',')
df2.head()

(103, 3)


,Postalcode,Borough,Neighbourhood
0,M1B\n,Scarborough\n,"Malvern , Rouge"
1,M1C\n,Scarborough\n,"Rouge Hill , Port Union , Highland Creek"
2,M1E\n,Scarborough\n,"Guildwood , Morningside , West Hill"
3,M1G\n,Scarborough\n,Woburn
4,M1H\n,Scarborough\n,Cedarbrae


# Part 2
##### Adding latitude and longitude.

In [54]:
!pip install geocoder

     |████████████████████████████████| 102kB 7.7MB/s ta 0:00:011


In [55]:
import geocoder


In [57]:
def get_latlng(postal_code):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Toronto, Ontario'.format(postal_code))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [61]:
postal_codes = df2['Postalcode']    
coords = [ get_latlng(postal_code) for postal_code in postal_codes.tolist() ]

In [75]:
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])
df2['Latitude'] = df_coords['Latitude']
df2['Longitude'] = df_coords['Longitude']
df2.head()

,Postalcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B\n,Scarborough\n,"Malvern , Rouge",43.808626,-79.189913
1,M1C\n,Scarborough\n,"Rouge Hill , Port Union , Highland Creek",43.785779,-79.157368
2,M1E\n,Scarborough\n,"Guildwood , Morningside , West Hill",43.765806,-79.185284
3,M1G\n,Scarborough\n,Woburn,43.771545,-79.218135
4,M1H\n,Scarborough\n,Cedarbrae,43.768791,-79.238813


In [65]:

df2[df2.Postalcode == 'M5G']

,Postalcode,Borough,Neighbourhood,Latitude,Longitude


In [77]:
df2.to_csv('task2.csv', index=False)

In [76]:
df2

,Postalcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B\n,Scarborough\n,"Malvern , Rouge",43.808626,-79.189913
1,M1C\n,Scarborough\n,"Rouge Hill , Port Union , Highland Creek",43.785779,-79.157368
2,M1E\n,Scarborough\n,"Guildwood , Morningside , West Hill",43.765806,-79.185284
3,M1G\n,Scarborough\n,Woburn,43.771545,-79.218135
4,M1H\n,Scarborough\n,Cedarbrae,43.768791,-79.238813
5,M1J\n,Scarborough\n,Scarborough Village,43.744203,-79.228725
6,M1K\n,Scarborough\n,"Kennedy Park , Ionview , East Birchmount Park",43.726881,-79.265694
7,M1L\n,Scarborough\n,"Golden Mile , Clairlea , Oakridge",43.713340,-79.284942
8,M1M\n,Scarborough\n,"Cliffside , Cliffcrest , Scarborough Village West",43.723538,-79.228353
9,M1N\n,Scarborough\n,"Birch Cliff , Cliffside West",43.696448,-79.265642


# Part 3
##### Adding Map of Toronto

In [69]:
!pip install folium
import numpy as np
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
import pandas as pd
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

import geocoder

     |████████████████████████████████| 92kB 5.4MB/s eta 0:00:011


In [78]:
TorontoData  = pd.read_csv('task2.csv')
TorontoData .head()

,Postalcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B\n,Scarborough\n,"Malvern , Rouge",43.808626,-79.189913
1,M1C\n,Scarborough\n,"Rouge Hill , Port Union , Highland Creek",43.785779,-79.157368
2,M1E\n,Scarborough\n,"Guildwood , Morningside , West Hill",43.765806,-79.185284
3,M1G\n,Scarborough\n,Woburn,43.771545,-79.218135
4,M1H\n,Scarborough\n,Cedarbrae,43.768791,-79.238813


In [79]:
toronto_map = folium.Map(location=[43.65, -79.4], zoom_start=12)

X = TorontoData['Latitude']
Y = TorontoData['Longitude']
Z = np.stack((X, Y), axis=1)

kmeans = KMeans(n_clusters=4, random_state=0).fit(Z)

clusters = kmeans.labels_
colors = ['red', 'green', 'blue', 'yellow']
TorontoData['Cluster'] = clusters

for latitude, longitude, borough, cluster in zip(TorontoData['Latitude'], TorontoData['Longitude'], TorontoData['Borough'], TorontoData['Cluster']):
    label = folium.Popup(borough, parse_html=True)
    folium.CircleMarker(
        [latitude, longitude],
        radius=5,
        popup=label,
        color='black',
        fill=True,
        fill_color=colors[cluster],
        fill_opacity=0.7).add_to(toronto_map)  

toronto_map

